# Qoperation objects
This notebook describes the concept of Qoperation in Quara and the related classes.

Quara supports operations such as State, Gate, Povm, etc.
These operations is called objects in Quara.
Two methods to generate objects.
- The typical objects can be generated from a module called `xxx_typical`. ("xxx" is a object name.)
- Users can also generate objects directly.


In [1]:
import numpy as np
np.set_printoptions(linewidth=200)

## Qoperation
Qoperation is the base class of these operations and has common properties and functions.

### common properties
The common properties are as follows:

- `is_physicality_required` - If this is True, this object must satisfy the physicality.
- `is_estimation_object` - If this is True, this object is for estimation.
- `on_para_eq_constraint` - If this is True, functions and algorithms work on the assumption that the parameters of this object satisfy the equality constraint. 
- `on_algo_eq_constraint` - If this is True, estimation algorithms use equality constraint. 
- `on_algo_ineq_constraint` - If this is True, estimation algorithms use inequality constraint. 
- `mode_proj_order` - the order in which the projection are performed, by default "eq_ineq".
   - "eq_ineq" - The projection are performed in the order of equality constraints, followed by inequality constraints.
   - "ineq_eq" - The projection are performed in the order of inequality constraints, followed by equality constraints.
- `eps_proj_physical` - Epsiron that is projection algorithm error threshold for being physical.
- `eps_truncate_imaginary_part` - The threshold to truncate imaginary part.

In [2]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.qoperation import QOperation

c_sys = generate_composite_system("qubit", 1)
qoperation = QOperation(c_sys)

print(f"is_physicality_required: {qoperation.is_physicality_required}")
print(f"is_estimation_object: {qoperation.is_estimation_object}")
print(f"on_para_eq_constraint: {qoperation.on_para_eq_constraint}")
print(f"on_algo_eq_constraint: {qoperation.on_algo_eq_constraint}")
print(f"on_algo_ineq_constraint: {qoperation.on_algo_ineq_constraint}")
print(f"mode_proj_order: {qoperation.mode_proj_order}")
print(f"eps_proj_physical: {qoperation.eps_proj_physical}")
print(f"eps_truncate_imaginary_part: {qoperation.eps_truncate_imaginary_part}")

is_physicality_required: True
is_estimation_object: True
on_para_eq_constraint: True
on_algo_eq_constraint: True
on_algo_ineq_constraint: True
mode_proj_order: eq_ineq
eps_proj_physical: 1e-14
eps_truncate_imaginary_part: None


### functions to check constraints
QOperation has functions to check constraints.

- `is_eq_constraint_satisfied()` - Whether the qoperation is satisfied equality constraints.
- `is_ineq_constraint_satisfied()` - Whether the qoperation is satisfied inequality constraints.
- `is_physical()` - Whether the qoperation is satisfied equality and inequality constraints.

`is_eq_constraint_satisfied()` and `is_ineq_constraint_satisfied()` are abstract functions and are implemented in a subclasses.

### projection functions
QOperation has projection functions.

- `calc_proj_eq_constraint()` - Calculates the projection of QOperation on equal constraint.
- `calc_proj_ineq_constraint()` - Calculates the projection of QOperation on inequal constraint.
- `calc_proj_physical()` - Calculates the projection of QOperation on equal and inequal constraint.
- `calc_proj_physical_with_var()` - Executes the algorithm of calc_proj_physical using variables. This functions is faster than calc_proj_physical.

`calc_proj_physical()` and `calc_proj_physical_with_var()` calculate the projection in the order specified by `mode_proj_order`.

### functions to transform parameters
QOperation has functions to transform parameters.

- `to_stacked_vector()` - Transforms all parameters to a one-dimensional vector (a numpy array).
- `to_var()` - Transforms parameters except for the part of the equality constraint to a one-dimensional vector (a numpy array).

These functions are abstract functions and are implemented in a subclasses.

### functions to generate special objects
QOperation has functions to generate special objects.

- `generate_zero_obj()` - Generates object with all parameters are zero.
- `generate_origin_obj()` - Generates origin object.

These functions are abstract functions and are implemented in a subclasses.

### supports arithmetic operations
Quara supports arithmetic operations between same type of QOperations.

- `__add__()` - Calculates `a + b` for Qoperation a, b.
- `__sub__()` - Calculates `a - b` for Qoperation a, b.
- `__mul__()` - Calculates `a * b` for Qoperation a and number b.
- `__rmul__()` - Calculates `a * b` for number a and Qoperation b.
- `__truediv__()` - Calculates `a / b` for Qoperation a and number b.

## Notation
$I$ is the identity matrix.  
$X := \begin{bmatrix} 0 & 1 \\ 1 & 0 \\ \end{bmatrix}$, $Y := \begin{bmatrix} 0 & -i \\ i & 0 \\ \end{bmatrix}$, and $Z := \begin{bmatrix} 1 & 0 \\ 0 & -1 \\ \end{bmatrix}$ are Pauli matrices.  
$B$ is a basis $\{ I/\sqrt{2}, X/\sqrt{2}, Y/\sqrt{2}, Z/\sqrt{2} \}$, which is normalized by Hilbert-Schmidt inner product $\langle S, T\rangle := \text{Tr}[ST]$.

## State
Quantum state (density matrix) can be represented by a linear combination of basis.
`vec` represents the coefficients of this linear combination in the form of a numpy array.  

Example.  
$z_0 = \begin{bmatrix} 1 & 0 \\ 0 & 0 \\ \end{bmatrix}$ can be represented by a linear combination of basis $1/\sqrt{2} \cdot I/\sqrt{2} + 0 \cdot X/\sqrt{2} + 0 \cdot Y/\sqrt{2} + 1/\sqrt{2} \cdot  Z/\sqrt{2}$. In this case `vec` of $z_0$ is $[ 1/\sqrt{2}, 0, 0, 1/\sqrt{2} ]$.

The methods for generating a State includes the following:

- Generate from `state_typical` module
- Generate State object directly

Generate from `state_typical` module by specifying CompositeSystem and state name (ex. "z0").

In [3]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.state_typical import generate_state_from_name

c_sys = generate_composite_system("qubit", 1)
state = generate_state_from_name(c_sys, "z0")
print(state)

Type:
State

Dim:
2

Vec:
[0.70710678 0.         0.         0.70710678]


Generate State object directly using CompositeSystem and a numpy array.

In [4]:
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import State

basis = get_normalized_pauli_basis(1)
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])
vec = np.array([1, 0, 0, 1]) / np.sqrt(2)
state = State(c_sys, vec)
print(state)

Type:
State

Dim:
2

Vec:
[0.70710678 0.         0.         0.70710678]


### specific properties
The property `vec` of State is a numpy array specified by the constructor argument `vec`.

In [5]:
state = State(c_sys, vec)
print(f"vec: {state.vec}")

vec: [0.70710678 0.         0.         0.70710678]


The property `dim` of State is a square root of the size of element of vec.

In [6]:
print(f"dim: {state.dim}")
print(f"square root of the size of element of vec: {int(np.sqrt(len(vec)))}")

dim: 2
square root of the size of element of vec: 2


### functions to check constraints
The `is_eq_constraint_satisfied()` function returns True, if and only if $\text{Tr}[\rho] = 1$, where $\rho$ is a density matrix of State.

In [7]:
print(f"is_eq_constraint_satisfied(): {state.is_eq_constraint_satisfied()}")
print(f"trace of density matrix: {np.trace(state.to_density_matrix())}")

is_eq_constraint_satisfied(): True
trace of density matrix: (0.9999999999999998+0j)


The `is_ineq_constraint_satisfied()` function returns True, if and only if $\rho$ is positive semidifinite matrix, where $\rho$ is a density matrix of State.

In [8]:
print(f"is_eq_constraint_satisfied(): {state.is_eq_constraint_satisfied()}")
print(f"is_positive_semidefinite(): {state.is_positive_semidefinite()}")

is_eq_constraint_satisfied(): True
is_positive_semidefinite(): True


### projection functions
`calc_proj_eq_constraint()` function calculates the projection of State on equal constraint.  
This function replaces the first element of `vec` with $1/\sqrt{d}$, where $d$ is `dim`.

In [9]:
vec = np.array([1.0, 2.0, 3.0, 4.0])
state = State(c_sys, vec, is_physicality_required=False)
proj_state = state.calc_proj_eq_constraint()
print(f"vec: {proj_state.vec}")

vec: [0.70710678 2.         3.         4.        ]


`calc_proj_ineq_constraint()` function calculates the projection of State on inequal constraint as follows:  

- Executes singular value decomposition on the density matrix $\rho$ of state, $\rho = U \Lambda U^{\dagger}$, where $\Lambda = \text{diag}[\lambda_0, \dots , \lambda_{d-1}]$, and $\lambda_{i} \in \mathbb{R}$.
- $\lambda^{\prime}_{i} := \begin{cases} \lambda_{i} & (\lambda_{i} \geq 0) \\ 0 & (\lambda_{i} < 0) \end{cases}$
- $\Lambda^{\prime} = \text{diag}[\lambda^{\prime}_0, \dots , \lambda^{\prime}_{d-1}]$
- $\rho^{\prime} = U \Lambda^{\prime} U^{\dagger}$
- The projection of State is $|\rho^{\prime} \rangle\rangle$.

In [10]:
vec = np.sqrt(2) * np.array([0, 0, 0, 1])
state = State(c_sys, vec, is_physicality_required=False)
print(f"density matrix before projection: \n{state.to_density_matrix()}")
proj_state = state.calc_proj_ineq_constraint()
print(f"density matrix after projection: \n{proj_state.to_density_matrix()}")
print(f"vec after projection: \n{proj_state.vec}")

density matrix before projection: 
[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]
density matrix after projection: 
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
vec after projection: 
[0.70710678 0.         0.         0.70710678]


### functions to transform parameters
`to_stacked_vector()` function returns a one-dimensional numpy array of all variables. This is equal to `vec`.


In [11]:
print(f"to_stacked_vector(): {state.to_stacked_vector()}")
print(f"vec: {state.vec}")

to_stacked_vector(): [0.         0.         0.         1.41421356]
vec: [0.         0.         0.         1.41421356]


If `on_para_eq_constraint` is True, then the first element of `vec` is equal to $1/\sqrt{d}$, where $d$ is `dim`. Thus, State is characterized by the second and subsequent elements of `vec`.  
Therefore, `to_var()` function returns the second and subsequent elements of `vec`, where `on_para_eq_constraint` is True.

In [12]:
vec = np.array([1, 0, 0, 1]) / np.sqrt(2)

In [13]:
# on_para_eq_constraint=True
state = State(c_sys, vec, on_para_eq_constraint=True)
print(f"to_var(): {state.to_var()}")

to_var(): [0.         0.         0.70710678]


In [14]:
# on_para_eq_constraint=False
state = State(c_sys, vec, on_para_eq_constraint=False)
print(f"to_var(): {state.to_var()}")

to_var(): [0.70710678 0.         0.         0.70710678]


### functions to generate special objects

In [15]:
zero_state = state.generate_zero_obj()
print(f"zero: {zero_state.vec}")
origin_state = state.generate_origin_obj()
print(f"origin: {origin_state.vec}")

zero: [0. 0. 0. 0.]
origin: [0.70710678 0.         0.         0.        ]


### supports arithmetic operations

In [16]:
vec1 = np.array([1.0, 2.0, 3.0, 4.0])
state1 = State(c_sys, vec1, is_physicality_required=False)
vec2 = np.array([5.0, 6.0, 7.0, 8.0])
state2 = State(c_sys, vec2, is_physicality_required=False)

print(f"sum: {(state1 + state2).vec}")
print(f"subtraction: {(state1 - state2).vec}")
print(f"right multiplication: {(2 * state1).vec}")
print(f"left multiplication: {(state1 * 2).vec}")
print(f"division: {(state1 / 2).vec}")

sum: [ 6.  8. 10. 12.]
subtraction: [-4. -4. -4. -4.]
right multiplication: [2. 4. 6. 8.]
left multiplication: [2. 4. 6. 8.]
division: [0.5 1.  1.5 2. ]


### calc_gradient functions
Calculates gradient of State with variable index.

In [17]:
grad_state = state.calc_gradient(0)
print(f"vec: {grad_state.vec}")

vec: [1. 0. 0. 0.]


### convert_basis function
Returns `vec` converted to the specified basis.

In [18]:
from quara.objects.matrix_basis import get_comp_basis

state = generate_state_from_name(c_sys, "z0")
converted_vec = state.convert_basis(get_comp_basis())
print(f"vec: {converted_vec}")

vec: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]


### to_density_matrix function
Returns density matrix of State. 

In [19]:
state = generate_state_from_name(c_sys, "z0")
print(f"to_density_matrix(): \n{state.to_density_matrix()}")
print(f"to_density_matrix_with_sparsity(): \n{state.to_density_matrix_with_sparsity()}")

to_density_matrix(): 
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
to_density_matrix_with_sparsity(): 
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


### some utility functions

In [20]:
print(f"is_trace_one(): {state.is_trace_one()}")
print(f"is_hermitian(): {state.is_hermitian()}")
print(f"is_positive_semidefinite(): {state.is_positive_semidefinite()}")
print(f"calc_eigenvalues(): {state.calc_eigenvalues()}")

is_trace_one(): True
is_hermitian(): True
is_positive_semidefinite(): True
calc_eigenvalues(): [0.9999999999999998, 0.0]


## state_typical
`generate_state_object_from_state_name_object_name()` function in `state_typical` module can easily generate objects related to State.  
The `generate_state_object_from_state_name_object_name()` function has the following arguments:

- The string that can be specified for `state_name` can be checked by executing the `get_state_names()` function. The tensor product of state_name "a", "b" is written "a_b".
- `object_name` can be the following string:
  - "pure_state_vector" - vector of pure state.
  - "density_mat" - density matrix.
  - "density_matrix_vector" - vectorized density matrix.
  - "state" - State object.
- `c_sys` - CompositeSystem of objects related to State. Specify when `object_name` is "density_matrix_vector" or "state".
- `is_physicality_required` - Whether the generated object is physicality required, by default True.

In [21]:
from quara.objects.state_typical import (
    get_state_names,
    generate_state_object_from_state_name_object_name,
)

print(f"get_state_names(): \n{get_state_names()}")

get_state_names(): 
['x0', 'x1', 'y0', 'y1', 'z0', 'z1', 'a', 'bell_phi_plus', 'bell_phi_minus', 'bell_psi_plus', 'bell_psi_minus', 'x0_x0', 'x0_x1', 'x0_y0', 'x0_y1', 'x0_z0', 'x0_z1', 'x0_a', 'x1_x0', 'x1_x1', 'x1_y0', 'x1_y1', 'x1_z0', 'x1_z1', 'x1_a', 'y0_x0', 'y0_x1', 'y0_y0', 'y0_y1', 'y0_z0', 'y0_z1', 'y0_a', 'y1_x0', 'y1_x1', 'y1_y0', 'y1_y1', 'y1_z0', 'y1_z1', 'y1_a', 'z0_x0', 'z0_x1', 'z0_y0', 'z0_y1', 'z0_z0', 'z0_z1', 'z0_a', 'z1_x0', 'z1_x1', 'z1_y0', 'z1_y1', 'z1_z0', 'z1_z1', 'z1_a', 'a_x0', 'a_x1', 'a_y0', 'a_y1', 'a_z0', 'a_z1', 'a_a', 'ghz', 'werner', 'x0_x0_x0', 'x0_x0_x1', 'x0_x0_y0', 'x0_x0_y1', 'x0_x0_z0', 'x0_x0_z1', 'x0_x0_a', 'x0_x1_x0', 'x0_x1_x1', 'x0_x1_y0', 'x0_x1_y1', 'x0_x1_z0', 'x0_x1_z1', 'x0_x1_a', 'x0_y0_x0', 'x0_y0_x1', 'x0_y0_y0', 'x0_y0_y1', 'x0_y0_z0', 'x0_y0_z1', 'x0_y0_a', 'x0_y1_x0', 'x0_y1_x1', 'x0_y1_y0', 'x0_y1_y1', 'x0_y1_z0', 'x0_y1_z1', 'x0_y1_a', 'x0_z0_x0', 'x0_z0_x1', 'x0_z0_y0', 'x0_z0_y1', 'x0_z0_z0', 'x0_z0_z1', 'x0_z0_a', 'x0_z1_x0

### object_name = "pure_state_vector"

In [22]:
vec = generate_state_object_from_state_name_object_name("z0", "pure_state_vector")
print(vec)

[1.+0.j 0.+0.j]


### object_name = "density_mat"

In [23]:
density_mat = generate_state_object_from_state_name_object_name("z0", "density_mat")
print(density_mat)

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


### object_name = "density_matrix_vector"

In [24]:
c_sys = generate_composite_system("qubit", 1)
vec = generate_state_object_from_state_name_object_name("z0", "density_matrix_vector", c_sys=c_sys)
print(vec)

[0.70710678 0.         0.         0.70710678]


### object_name = "state"

In [25]:
c_sys = generate_composite_system("qubit", 1)
state = generate_state_object_from_state_name_object_name("z0", "state", c_sys=c_sys)
print(state)

Type:
State

Dim:
2

Vec:
[0.70710678 0.         0.         0.70710678]


## Povm
Quantum measurement has two mathematical treatments. One is positive operator-valued measure (POVM), which can describe the effect of quantum measurement on the probability distribution of its measurement outcome only. The other is measurement apparatus, which can describe both of the effect on the probability distribution and states after the measurement. Current version of quara prepare a class for POVM only. A class for measurement apparatus will be added in the near future.

Povm can be represented by a list of POVM element as `vecs`.  
Each emelemts is a linear combination of basis and represents the coefficients of this linear combination in the form of a numpy array.

Example:  
Povm elements $\Pi_0 = \begin{bmatrix} 1 & 0 \\ 0 & 0 \\ \end{bmatrix}$, $\Pi_1 = \begin{bmatrix} 0 & 0 \\ 0 & 1 \\ \end{bmatrix}$.  
Povm $\Pi = \{ \Pi_x \}_{x=0,1}$. index $x$ is called outcomes.  
We can write $\Pi_0= 1/\sqrt{2} \cdot I/\sqrt{2} + 0 \cdot X/\sqrt{2} + 0 \cdot Y/\sqrt{2} + 1/\sqrt{2} \cdot  Z/\sqrt{2}$ and  $\Pi_1= 1/\sqrt{2} \cdot I/\sqrt{2} + 0 \cdot X/\sqrt{2} + 0 \cdot Y/\sqrt{2} - 1/\sqrt{2} \cdot  Z/\sqrt{2}$.  
In this case `vecs` of $\Pi$ is a list of $[ 1/\sqrt{2}, 0, 0, 1/\sqrt{2} ]$ and $[ 1/\sqrt{2}, 0, 0, -1/\sqrt{2} ]$.

The methods for generating a State includes the following:

- Generate from `povm_typical` module
- Generate Povm object directly

Generate from `povm_typical` module by specifying CompositeSystem and povm name (ex. "z").

In [26]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.povm_typical import generate_povm_from_name

c_sys = generate_composite_system("qubit", 1)
povm = generate_povm_from_name("z", c_sys)
print(povm)

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]


Generate Povm object directly using CompositeSystem and a list of numpy array.

In [27]:
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import Povm

basis = get_normalized_pauli_basis(1)
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])
vec1 = np.array([1, 0, 0, 1]) / np.sqrt(2)
vec2 = np.array([1, 0, 0, -1]) / np.sqrt(2)
vecs = [vec1, vec2]
povm = Povm(c_sys, vecs)
print(povm)

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]


### specific properties
The property `vecs` of Povm is a list of a numpy array specified by the constructor argument `vecs`.

In [28]:
povm = Povm(c_sys, vecs)
print(f"vecs: \n{povm.vecs}")

vecs: 
(array([0.70710678, 0.        , 0.        , 0.70710678]), array([ 0.70710678,  0.        ,  0.        , -0.70710678]))


The property `dim` of Povm is a square root of the size of element of `vecs`.

In [29]:
print(f"dim: {povm.dim}")
print(f"square root of the size of element of vecs: {int(np.sqrt(len(vecs[0])))}")

dim: 2
square root of the size of element of vecs: 2


The property `num_outcomes` of Povm is the number of POVM elements.

In [30]:
print(f"num_outcomes: {povm.num_outcomes}")
print(f"number of POVM elements: {len(vecs)}")

num_outcomes: 2
number of POVM elements: 2


### functions to check constraints
The `is_eq_constraint_satisfied()` function returns True, if and only if $\sum_x \Pi_x = I$, where $\Pi_x$ are Povm elements.

In [31]:
from operator import add
from functools import reduce

print(f"is_eq_constraint_satisfied(): {povm.is_eq_constraint_satisfied()}")
print(f"sum of matrices: \n{reduce(add, povm.matrices())}")

is_eq_constraint_satisfied(): True
sum of matrices: 
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


The `is_ineq_constraint_satisfied()` function returns True, if and only if all $\Pi_x$ are positive semidifinite matrices.

In [32]:
print(f"is_eq_constraint_satisfied(): {povm.is_eq_constraint_satisfied()}")
print(f"matrix(0): \n{povm.matrix(0)}")
print(f"matrix(1): \n{povm.matrix(1)}")

is_eq_constraint_satisfied(): True
matrix(0): 
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
matrix(1): 
[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


### projection functions
`calc_proj_eq_constraint()` function calculates the projection of Povm on equal constraint.  
Let `num_outcomes` = $m$, `dim` = $d$, and $\tilde{I} = I/\sqrt{d}$. Then $\sum_{x=0}^{m-1} \Pi_x = I = \sqrt{d} \cdot \tilde{I}$.  
Therefore $\sum_{x=0}^{m-1} |\Pi_x\rangle\rangle = |\tilde{I}\rangle\rangle = [ \sqrt{d}, 0, \dots, 0 ]^T$. Thus, the last element of Povm $\Pi_{m-1}$ can be determined, depending on other elements, as follows:  

- $|\Pi_x\rangle\rangle = [a_{x,0}, \dots, a_{x,d^2-1}]^T$
- $\bar{a_\alpha} := \frac{1}{m} \sum_{x=0}^{m-1} (|\Pi_x\rangle\rangle)_\alpha$, where $\alpha = 0, \dots, d^2-1$.
- $c_\alpha := \begin{cases} \sqrt{d}/m & (\alpha = 0) \\ 0 & (\alpha = 1, \dots, d^2-1) \end{cases}$
- $a_{x,\alpha}^\prime := a_{x,\alpha} - \bar{a_\alpha} + c_\alpha$
- The projection of Povm  $|\tilde{\Pi}_x\rangle\rangle$ is $[ a_{x,0}^\prime, \dots, a_{x,d^2-1}^\prime ]^T$.

This function replaces the last element of `vecs` with $1/\sqrt{d}$, where $d$ is `dim`.

In [33]:
vec1 = np.array([1.0, 2.0, 3.0, 4.0])
vec2 = np.array([5.0, 6.0, 7.0, 8.0])
vecs = [vec1, vec2]

In [34]:
povm = Povm(c_sys, vecs, is_physicality_required=False)
proj_povm = povm.calc_proj_eq_constraint()
print(f"vecs: \n{proj_povm.vecs}")

vecs: 
(array([-1.29289322, -2.        , -2.        , -2.        ]), array([2.70710678, 2.        , 2.        , 2.        ]))


`calc_proj_ineq_constraint()` function calculates the projection of Povm $\{|\Pi_x\rangle\rangle\}_x$ on inequal constraint as follows:

- For each $x$, calculate the following:
  - Executes singular value decomposition on the elements of Povm $\Pi_x$, $\Pi_x = U \Lambda U^{\dagger}$, where $\Lambda = \text{diag}[\lambda_0, \dots , \lambda_{d-1}]$, and $\lambda_{i} \in \mathbb{R}$.
  - $\lambda^{\prime}_{i} := \begin{cases} \lambda_{i} & (\lambda_{i} \geq 0) \\ 0 & (\lambda_{i} < 0) \end{cases}$
  - $\Lambda^{\prime} = \text{diag}[\lambda^{\prime}_0, \dots , \lambda^{\prime}_{d-1}]$
  - $\Pi_x^{\prime} = U \Lambda^{\prime} U^{\dagger}$
- The projection of Povm is $\{|\Pi_x^{\prime} \rangle\rangle\}_x$.

In [35]:
povm = Povm(c_sys, vecs, is_physicality_required=False)
proj_povm = povm.calc_proj_ineq_constraint()
print(f"vecs: \n{proj_povm.vecs}")

vecs: 
(array([3.1925824 , 1.18569534, 1.77854301, 2.37139068]), array([8.60327781, 4.22884788, 4.93365586, 5.63846384]))


### functions to transform parameters
`to_stacked_vector()` function returns a one-dimensional numpy array of all variables. This is equal to a concatenated vector of elements of `vecs`.

In [36]:
print(f"to_stacked_vector(): {povm.to_stacked_vector()}")
print(f"vecs: \n{povm.vecs}")

to_stacked_vector(): [1. 2. 3. 4. 5. 6. 7. 8.]
vecs: 
(array([1., 2., 3., 4.]), array([5., 6., 7., 8.]))


If Povm $\Pi = \{ \Pi_0, \dots , \Pi_{m-1} \}$ and `on_para_eq_constraint` is True, then the last element of Povm $\Pi_{m-1}$ is equal to $I - \sum_{x=0}^{m-2} \Pi_x$. Thus, Povm is characterized by `vecs[0]`,..., `vecs[m-2]`.  
Therefore, `to_var()` function returns a vector combining elements of of `vecs[0]`,..., `vecs[m-2]`, where `on_para_eq_constraint` is True.

In [37]:
vec1 = np.array([1, 0, 0, 1]) / np.sqrt(2)
vec2 = np.array([1, 0, 0, -1]) / np.sqrt(2)
vecs = [vec1, vec2]

In [38]:
# on_para_eq_constraint=True
povm = Povm(c_sys, vecs, on_para_eq_constraint=True)
print(f"to_var(): {povm.to_var()}")

to_var(): [0.70710678 0.         0.         0.70710678]


In [39]:
# on_para_eq_constraint=False
povm = Povm(c_sys, vecs, on_para_eq_constraint=False)
print(f"to_var(): {povm.to_var()}")

to_var(): [ 0.70710678  0.          0.          0.70710678  0.70710678  0.          0.         -0.70710678]


### functions to generate special objects

In [40]:
zero_povm = povm.generate_zero_obj()
print(f"zero: \n{zero_povm.vecs}")
origin_povm = povm.generate_origin_obj()
print(f"origin: \n{origin_povm.vecs}")

zero: 
(array([0., 0., 0., 0.]), array([0., 0., 0., 0.]))
origin: 
(array([0.70710678, 0.        , 0.        , 0.        ]), array([0.70710678, 0.        , 0.        , 0.        ]))


### supports arithmetic operations

In [41]:
vec11 = np.array([1.0, 2.0, 3.0, 4.0])
vec12 = np.array([5.0, 6.0, 7.0, 8.0])
povm1 = Povm(c_sys, [vec11, vec12], is_physicality_required=False)
vec21 = np.array([9.0, 10.0, 11.0, 12.0])
vec22 = np.array([13.0, 14.0, 15.0, 16.0])
povm2 = Povm(c_sys, [vec21, vec22], is_physicality_required=False)

print(f"sum: \n{(povm1 + povm2).vecs}")
print(f"subtraction: \n{(povm1 - povm2).vecs}")
print(f"right multiplication: \n{(2 * povm1).vecs}")
print(f"left multiplication: \n{(povm1 * 2).vecs}")
print(f"division: \n{(povm1 / 2).vecs}")

sum: 
(array([10., 12., 14., 16.]), array([18., 20., 22., 24.]))
subtraction: 
(array([-8., -8., -8., -8.]), array([-8., -8., -8., -8.]))
right multiplication: 
(array([2., 4., 6., 8.]), array([10., 12., 14., 16.]))
left multiplication: 
(array([2., 4., 6., 8.]), array([10., 12., 14., 16.]))
division: 
(array([0.5, 1. , 1.5, 2. ]), array([2.5, 3. , 3.5, 4. ]))


### calc_gradient functions
Calculates gradient of Povm with variable index.

In [42]:
grad_povm = povm.calc_gradient(0)
print(f"vecs: {grad_povm.vecs}")

vecs: (array([1., 0., 0., 0.]), array([0., 0., 0., 0.]))


### convert_basis function
Returns `vecs` converted to the specified basis.

In [43]:
from quara.objects.matrix_basis import get_comp_basis

povm = generate_povm_from_name("z", c_sys)
converted_vecs = povm.convert_basis(get_comp_basis())
print(f"vecs: {converted_vecs}")

vecs: [array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]), array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j])]


### generate_mprocess function
Generates MProcess from this Povm.

In [44]:
print(povm.generate_mprocess())

16it [00:00, 355.56it/s]Type:
MProcess

Dim:
2

HSs:
[array([[0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5]]), array([[ 0.5,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [-0.5,  0. ,  0. ,  0.5]])]

ModeSampling:
False



### some utility functions

In [45]:
print(f"vec(0): \n{povm.vec(0)}")
print(f"matrices(): \n{povm.matrices()}")
print(f"matrix(0): \n{povm.matrix(0)}")
print(f"is_hermitian(): \n{povm.is_hermitian()}")
print(f"is_positive_semidefinite(): \n{povm.is_positive_semidefinite()}")
print(f"is_identity_sum(): \n{povm.is_identity_sum()}")
print(f"calc_eigenvalues(): \n{povm.calc_eigenvalues()}")

vec(0): 
[0.70710678 0.         0.         0.70710678]
matrices(): 
[matrix([[1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j]]), matrix([[0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]])]
matrix(0): 
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
is_hermitian(): 
True
is_positive_semidefinite(): 
True
is_identity_sum(): 
True
calc_eigenvalues(): 
[[0.9999999999999998, 0.0], [0.9999999999999998, 0.0]]


## povm_typical
`generate_povm_object_from_povm_name_object_name()` function in `povm_typical` module can easily generate objects related to Povm.  
The `generate_povm_object_from_povm_name_object_name()` function has the following arguments:

- The string that can be specified for `povm_name` can be checked by executing the get_povm_names function. The tensor product of povm_name "a", "b" is written "a_b".
- `object_name` can be the following string:
  - "pure_state_vectors" - list of vector of pure states.
  - "matrices" - matrices of vector of pure states.
  - "vectors" - list of vectorized matrices.
  - "povm" - Povm object.
- `c_sys` - CompositeSystem of objects related to Povm. Specify when `object_name` is "povm".
- `basis` - MatrixBasis of objects related to Povm. Specify when `object_name` is "vectors".
- `is_physicality_required` - Whether the generated object is physicality required, by default True.

In [46]:
from quara.objects.povm_typical import (
    get_povm_names,
    generate_povm_object_from_povm_name_object_name,
)

print(f"get_povm_names(): \n{get_povm_names()}")

get_povm_names(): 
['x', 'y', 'z', 'bell', 'x_x', 'x_y', 'x_z', 'y_x', 'y_y', 'y_z', 'z_x', 'z_y', 'z_z', 'x_x_x', 'x_x_y', 'x_x_z', 'x_y_x', 'x_y_y', 'x_y_z', 'x_z_x', 'x_z_y', 'x_z_z', 'y_x_x', 'y_x_y', 'y_x_z', 'y_y_x', 'y_y_y', 'y_y_z', 'y_z_x', 'y_z_y', 'y_z_z', 'z_x_x', 'z_x_y', 'z_x_z', 'z_y_x', 'z_y_y', 'z_y_z', 'z_z_x', 'z_z_y', 'z_z_z', '01x3', '01y3', 'z3', 'z2', '02x3', '02y3', '12x3', '12y3', '01x3_01x3', '01x3_01y3', '01x3_z3', '01x3_z2', '01x3_02x3', '01x3_02y3', '01x3_12x3', '01x3_12y3', '01y3_01x3', '01y3_01y3', '01y3_z3', '01y3_z2', '01y3_02x3', '01y3_02y3', '01y3_12x3', '01y3_12y3', 'z3_01x3', 'z3_01y3', 'z3_z3', 'z3_z2', 'z3_02x3', 'z3_02y3', 'z3_12x3', 'z3_12y3', 'z2_01x3', 'z2_01y3', 'z2_z3', 'z2_z2', 'z2_02x3', 'z2_02y3', 'z2_12x3', 'z2_12y3', '02x3_01x3', '02x3_01y3', '02x3_z3', '02x3_z2', '02x3_02x3', '02x3_02y3', '02x3_12x3', '02x3_12y3', '02y3_01x3', '02y3_01y3', '02y3_z3', '02y3_z2', '02y3_02x3', '02y3_02y3', '02y3_12x3', '02y3_12y3', '12x3_01x3', '12x3_01y3

### object_name = "pure_state_vectors"

In [47]:
vecs = generate_povm_object_from_povm_name_object_name("z", "pure_state_vectors")
print(vecs)

[array([1.+0.j, 0.+0.j]), array([0.+0.j, 1.+0.j])]


### object_name = "matrices"

In [48]:
matrices = generate_povm_object_from_povm_name_object_name("z", "matrices")
print(matrices)

[array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]]), array([[0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])]


### object_name = "vectors"

In [49]:
basis = get_normalized_pauli_basis(1)
vectors = generate_povm_object_from_povm_name_object_name("z", "vectors", basis=basis)
print(vectors)

[array([0.70710678, 0.        , 0.        , 0.70710678]), array([ 0.70710678,  0.        ,  0.        , -0.70710678])]


### object_name = "povm"

In [50]:
c_sys = generate_composite_system("qubit", 1)
povm = generate_povm_object_from_povm_name_object_name("z", "povm", c_sys=c_sys)
print(povm)

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]


## Gate
Mathematically a quantum gate is a linear trace-preserving and completely positive (L-TPCP) map on the space of quantum states, and there are several different matrix representations for quantum gate. In Quara, a class Gate is based on the Hilbert-Schmidt matrix representation of a gate with respect to the matrix basis in the CompositeSystem.  
This matrix representation is denote `hs` in Quara. `hs` is a 2-dimensional numpy array.  

Example.  
$X$ maps each element of basis as follows
$X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \\ \end{bmatrix}$ maps each element of basis $B = \{ I/\sqrt{2},  X/\sqrt{2},  Y/\sqrt{2},  Z/\sqrt{2} \}$ as follows:

- $I/\sqrt{2} \mapsto X \cdot I/\sqrt{2} \cdot X^\dagger = I/\sqrt{2} = [1, 0, 0, 0]^T$ on basis $B$.
- $X/\sqrt{2} \mapsto X \cdot X/\sqrt{2} \cdot X^\dagger = X/\sqrt{2} = [0, 1, 0, 0]^T$ on basis $B$.
- $Y/\sqrt{2} \mapsto X \cdot Y/\sqrt{2} \cdot X^\dagger = -Y/\sqrt{2} = [0, 0, -1, 0]^T$ on basis $B$.
- $Z/\sqrt{2} \mapsto X \cdot Z/\sqrt{2} \cdot X^\dagger = -Z/\sqrt{2} = [0, 0, 0, -1]^T$ on basis $B$.

Therefore, the Hilbert-Schmidt matrix representation `hs` of $X$ is $\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & -1 & 0 \\ 0 & 0 & 0 & -1 \end{bmatrix}$.

The methods for generating a Gate includes the following:

- Generate from `gate_typical` module
- Generate Gate object directly

Generate from `gate_typical` module by specifying CompositeSystem and gate name (ex. "x").

In [51]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.gate_typical import generate_gate_from_gate_name

c_sys = generate_composite_system("qubit", 1)
gate = generate_gate_from_gate_name("x", c_sys)
print(gate)

16it [00:00, 197.53it/s]Type:
Gate

Dim:
2

HS:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]



Generate Gate object directly using CompositeSystem and a numpy array.

In [52]:
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.gate import Gate

basis = get_normalized_pauli_basis(1)
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])
hs = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]], dtype=np.float64)
gate = Gate(c_sys, hs)
print(gate)

16it [00:00, 207.80it/s]Type:
Gate

Dim:
2

HS:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]



### specific properties
The property `hs` of Gate is a 2-dimensional numpy array specified by the constructor argument `hs`.

In [53]:
gate = Gate(c_sys, hs)
print(f"hs: \n{gate.hs}")

hs: 
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]


The property `dim` of Gate is the size of square matrix `hs`.

In [54]:
print(f"dim: {gate.dim}")
print(f"size of square matrix hs: {int(np.sqrt(hs.shape[0]))}")

dim: 2
size of square matrix hs: 2


### functions to check constraints
The `is_eq_constraint_satisfied()` function returns True, if and only if `hs` is TP(trace-preserving map), i.e. if and only if the first row of `hs` is equal to $[ 1, 0, \dots, 0 ]$.

In [55]:
print(f"is_eq_constraint_satisfied(): {gate.is_eq_constraint_satisfied()}")
print(f"is_tp(): {gate.is_tp()}")
print(f"hs[0]: {gate.hs[0]}")

is_eq_constraint_satisfied(): True
is_tp(): True
hs[0]: [1. 0. 0. 0.]


The `is_ineq_constraint_satisfied()` function returns True, if and only if `hs` is CP(Complete-Positivity-Preserving), i.e. if and only if Choi matrix of `hs` is positive semidifinite matrix.

In [56]:
print(f"is_ineq_constraint_satisfied(): {gate.is_ineq_constraint_satisfied()}")
print(f"is_cp(): {gate.is_cp()}")

is_ineq_constraint_satisfied(): True
is_cp(): True


### projection functions
`calc_proj_eq_constraint()` function calculates the projection of Gate on equal constraint.  
This function replaces the first row of `hs` with $[ 1, 0, \dots, 0 ]$.

In [57]:
hs = np.array(range(16), dtype=np.float64).reshape((4, 4))
print(f"hs: \n{gate.hs}")

hs: 
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]


In [58]:
gate = Gate(c_sys, hs, is_physicality_required=False)
proj_gate = gate.calc_proj_eq_constraint()
print(f"hs: \n{proj_gate.hs}")

hs: 
[[ 1.  0.  0.  0.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]


`calc_proj_ineq_constraint()` function calculates the projection of Gate with `hs` on inequal constraint as follows:

- Let $\text{Choi}$ be Choi matrix of `hs`
- Executes singular value decomposition on $\text{Choi}$, $\text{Choi} = U \Lambda U^{\dagger}$, where $\Lambda = \text{diag}[\lambda_0, \dots , \lambda_{d-1}]$, and $\lambda_{i} \in \mathbb{R}$.
- $\lambda^{\prime}_{i} := \begin{cases} \lambda_{i} & (\lambda_{i} \geq 0) \\ 0 & (\lambda_{i} < 0) \end{cases}$
- $\Lambda^{\prime} = \text{diag}[\lambda^{\prime}_0, \dots , \lambda^{\prime}_{d-1}]$
- $\text{Choi}^{\prime} = U \Lambda^{\prime} U^{\dagger}$
- Let $\text{HS}^{\prime}$ be Hilbert-Schmidt matrix representation of $\text{Choi}^{\prime}$
- The projection of Gate is Gate with `hs` = $\text{HS}^{\prime}$.

In [59]:
gate = Gate(c_sys, hs, is_physicality_required=False)
proj_gate = gate.calc_proj_ineq_constraint()
print(f"hs: \n{proj_gate.hs}")

hs: 
[[15.84558996  4.43570942  5.29265833  6.14960724]
 [ 2.63097854  2.34702553  3.08437192  3.44443746]
 [ 4.98440409  4.50900796  4.73510284  5.71575945]
 [ 7.33782964  6.29370952  7.14039548  7.60980059]]


### functions to transform parameters
`to_stacked_vector()` function returns a one-dimensional numpy array of all variables. This is equal to flattened `hs`.

In [60]:
print(f"to_stacked_vector(): {gate.to_stacked_vector()}")
print(f"lattened hs: {gate.hs.flatten()}")

to_stacked_vector(): [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
lattened hs: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]


If `on_para_eq_constraint` is True, then the first row of `hs` is equal to $[1, 0, \dots, 0]$. Thus, Gate is characterized by the second and subsequent rows of `hs`.  
Therefore, `to_var()` function returns the flattened second and subsequent rows of `hs`, where `on_para_eq_constraint` is True.

In [61]:
# on_para_eq_constraint=True
gate = Gate(c_sys, hs, is_physicality_required=False, on_para_eq_constraint=True)
print(f"to_var(): {gate.to_var()}")

to_var(): [ 4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]


In [62]:
# on_para_eq_constraint=False
gate = Gate(c_sys, hs, is_physicality_required=False, on_para_eq_constraint=False)
print(f"to_var(): {gate.to_var()}")

to_var(): [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]


### functions to generate special objects

In [63]:
zero_gate = gate.generate_zero_obj()
print(f"zero: \n{zero_gate.hs}")
origin_gate = gate.generate_origin_obj()
print(f"origin: \n{origin_gate.hs}")

zero: 
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
origin: 
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### supports arithmetic operations

In [64]:
hs1 = np.array(range(16), dtype=np.float64).reshape((4, 4))
gate1 = Gate(c_sys, hs1, is_physicality_required=False)
hs2 = np.array(range(16, 32), dtype=np.float64).reshape((4, 4))
gate2 = Gate(c_sys, hs2, is_physicality_required=False)

print(gate1.hs)
print(gate2.hs)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]
[[16. 17. 18. 19.]
 [20. 21. 22. 23.]
 [24. 25. 26. 27.]
 [28. 29. 30. 31.]]


In [65]:
print(f"sum: \n{(gate1 + gate2).hs}")
print(f"subtraction: \n{(gate1 - gate2).hs}")
print(f"right multiplication: \n{(2 * gate1).hs}")
print(f"left multiplication: \n{(gate1 * 2).hs}")
print(f"division: \n{(gate1 / 2).hs}")

sum: 
[[16. 18. 20. 22.]
 [24. 26. 28. 30.]
 [32. 34. 36. 38.]
 [40. 42. 44. 46.]]
subtraction: 
[[-16. -16. -16. -16.]
 [-16. -16. -16. -16.]
 [-16. -16. -16. -16.]
 [-16. -16. -16. -16.]]
right multiplication: 
[[ 0.  2.  4.  6.]
 [ 8. 10. 12. 14.]
 [16. 18. 20. 22.]
 [24. 26. 28. 30.]]
left multiplication: 
[[ 0.  2.  4.  6.]
 [ 8. 10. 12. 14.]
 [16. 18. 20. 22.]
 [24. 26. 28. 30.]]
division: 
[[0.  0.5 1.  1.5]
 [2.  2.5 3.  3.5]
 [4.  4.5 5.  5.5]
 [6.  6.5 7.  7.5]]


### calc_gradient functions
Calculates gradient of Gate with variable index.

In [66]:
grad_gate = gate.calc_gradient(0)
print(f"hs: \n{grad_gate.hs}")

hs: 
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### convert_basis function
Returns `hs` converted to the specified basis.

In [67]:
from quara.objects.matrix_basis import get_comp_basis

gate = generate_gate_from_gate_name("x", c_sys)
converted_hs = gate.convert_basis(get_comp_basis())
print(f"hs: \n{converted_hs}")

hs: 
[[-2.23711432e-17+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j  1.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  1.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  1.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 1.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j -2.23711432e-17+0.j]]


### to_choi_matrix
Returns Choi matrix of Gate.

In [68]:
gate = generate_gate_from_gate_name("x", c_sys)
print(f"to_choi_matrix(): \n{gate.to_choi_matrix()}")
print(f"to_choi_matrix_with_dict(): \n{gate.to_choi_matrix_with_dict()}")
print(f"to_choi_matrix_with_sparsity(): \n{gate.to_choi_matrix_with_sparsity()}")

to_choi_matrix(): 
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
to_choi_matrix_with_dict(): 
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
to_choi_matrix_with_sparsity(): 
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


### to_kraus_matrices
Returns Kraus matrices of Gate.

In [69]:
gate = generate_gate_from_gate_name("x", c_sys)
print(f"to_kraus_matrices(): \n{gate.to_kraus_matrices()}")

to_kraus_matrices(): 
[array([[0.+0.j, 1.+0.j],
       [1.+0.j, 0.+0.j]])]


### to_process_matrix
Returns process matrix of Gate.

In [70]:
gate = generate_gate_from_gate_name("x", c_sys)
print(f"to_process_matrix(): \n{gate.to_process_matrix()}")

to_process_matrix(): 
[[-2.23711432e-17+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  1.00000000e+00+0.j  1.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  1.00000000e+00+0.j  1.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j -2.23711432e-17+0.j]]


### some utility functions

In [71]:
print(f"is_tp(): {gate.is_tp()}")
print(f"is_cp(): {gate.is_cp()}")

is_tp(): True
is_cp(): True


## gate_typical
`generate_gate_object_from_gate_name_object_name()` function in `gate_typical` module can easily generate objects related to Gate.  
The `generate_gate_object_from_gate_name_object_name()` function has the following arguments:

- The string that can be specified for `gate_name` can be checked by executing the `get_gate_names()` function. The tensor product of state_name "a", "b" is written "a_b".
- `object_name` can be the following string:
  - "unitary_mat" - unitary matrix of the gate.
  - "gate_mat" - The Hilbert-Schmidt matrix representation of the gate.
  - "gate" - Gate object.
- `c_sys` - CompositeSystem of objects related to Gate. Specify when `object_name` is "gate".
- `is_physicality_required` - Whether the generated object is physicality required, by default True.

In [72]:
from quara.objects.gate_typical import (
    get_gate_names,
    generate_gate_object_from_gate_name_object_name,
)

print(f"get_gate_names(): \n{get_gate_names()}")

get_gate_names(): 
['identity', 'x90', 'x180', 'x', 'y90', 'y180', 'y', 'z90', 'z180', 'z', 'phase', 'phase_daggered', 'piover8', 'piover8_daggered', 'hadamard', 'cx', 'cz', 'swap', 'zx90', 'zz90', 'toffoli', 'fredkin', '01x90', '01y90', '01z90', '12x90', '12y90', '12z90', '02x90', '02y90', '02z90', '01x180', '01y180', '01z180', '12x180', '12y180', '12z180', '02x180', '02y180', '02z180', 'i01x90', 'i01x180', 'i01y90', 'i01y180', 'i01z90', 'i01z180', 'i12x90', 'i12x180', 'i12y90', 'i12y180', 'i12z90', 'i12z180', 'i02x90', 'i02x180', 'i02y90', 'i02y180', 'i02z90', 'i02z180', '01xi90', '01xi180', '01x01x90', '01x01x180', '01x01y90', '01x01y180', '01x01z90', '01x01z180', '01x12x90', '01x12x180', '01x12y90', '01x12y180', '01x12z90', '01x12z180', '01x02x90', '01x02x180', '01x02y90', '01x02y180', '01x02z90', '01x02z180', '01yi90', '01yi180', '01y01x90', '01y01x180', '01y01y90', '01y01y180', '01y01z90', '01y01z180', '01y12x90', '01y12x180', '01y12y90', '01y12y180', '01y12z90', '01y12z180', '01

### object_name = "unitary_mat"

In [73]:
mat = generate_gate_object_from_gate_name_object_name("x", "unitary_mat")
print(mat)

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]


### object_name = "gate_mat"

In [74]:
mat = generate_gate_object_from_gate_name_object_name("x", "gate_mat")
print(mat)

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]


### object_name = "gate"

In [75]:
c_sys = generate_composite_system("qubit", 1)
gate = generate_gate_object_from_gate_name_object_name("x", "gate", c_sys=c_sys)
print(gate)

16it [00:00, 380.94it/s]Type:
Gate

Dim:
2

HS:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]



## EffectiveLindbladian

Assume the following situation:

- $B = \{ B_\alpha \}$ is a orthonormalized Hermitian matrix basis with $B_0 = I/\sqrt{d}$, where $d$ is dimension of CompositeSystem.
- $\rho$ is a density matrix.
- $H = \sum_{\alpha=1}^{d^2-1} H_\alpha B_\alpha$ ($H_\alpha \in \mathbb{R}$, $H \in \mathbb{C}^{d \times d}$) : Hermitian matrix.
- $J = \sum_{\alpha=0}^{d^2-1} J_\alpha B_\alpha$ ($J_\alpha \in \mathbb{R}$, $J \in \mathbb{C}^{d \times d}$) : Hermitian matrix.
- $K$ is a Hermitian and positive semidifinite matrix, with $(\alpha, \beta)$ entries are $K_{\alpha, \beta}$. ($K_{\alpha, \beta} \in \mathbb{R}$,  $K \in \mathbb{C}^{(d^2-1) \times (d^2-1)}$))

Then Lindbladian operator $\mathcal{L}$ is defined by $\mathcal{L}(\rho) = -i[H, \rho] + \{J, \rho\} + \sum_{\alpha, \beta=1}^{d^2-1} K_{\alpha, \beta} B_\alpha \rho B^{\dagger}_\beta$.  
Let $L^{cb}$ (resp. $L^{gb}$) be a Hilbert-Schmidt representation matrix of $\mathcal{L}$ on computational basis (resp. on basis $B$).  
We can write $L^{cb} = -i(H \otimes I - I \otimes \bar{H}) + (J \otimes I + I \otimes \bar{J}) + \sum_{\alpha, \beta=1}^{d^2-1} K_{\alpha, \beta} B_\alpha \otimes \overline{B_\beta}$. And $L^{gb} \in \mathbb{R}^{d^2 \times d^2}$.  
The property `hs` of EffectiveLindbladian is a two-dimensional numpy array $L^{gb}$ in Quara.  
Also, there is a relationship $G = e^L$, where $G$ is a `hs` of Gate and $e^L$ is a exponential of `hs` of EffectiveLindbladian.  
The following terms are used in Quara:

- $-i(H \otimes I - I \otimes \bar{H})$ is called H-part.
- $J \otimes I + I \otimes \bar{J}$ is called J-part.
- $\sum_{\alpha, \beta=1}^{d^2-1} K_{\alpha, \beta} B_\alpha \otimes \overline{B_\beta}$ is called K-part.
- The sum of J-part and K-part is called D-part.

EffectiveLindbladian class inherits Gate class in Quara.

Generate from `effective_lindbladian_typical` module by specifying CompositeSystem and state mprocess (ex. “z-type1”).

In [76]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.effective_lindbladian_typical import generate_effective_lindbladian_from_gate_name

c_sys = generate_composite_system("qubit", 1)
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(el)

Type:
EffectiveLindbladian

Dim:
2

HS:
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         -3.14159265]
 [ 0.          0.          3.14159265  0.        ]]


Generate EffectiveLindbladian object directly using CompositeSystem and a numpy array.

In [77]:
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.effective_lindbladian import EffectiveLindbladian

basis = get_normalized_pauli_basis(1)
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])
hs = np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, -np.pi], [0, 0, np.pi, 0]], dtype=np.float64)
el = EffectiveLindbladian(c_sys, hs)
print(el)

Type:
EffectiveLindbladian

Dim:
2

HS:
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         -3.14159265]
 [ 0.          0.          3.14159265  0.        ]]


### specific properties
The property `hs` of EffectiveLindbladian is a 2-dimensional numpy array specified by the constructor argument `hs`

In [78]:
el = EffectiveLindbladian(c_sys, hs)
print(f"hs: \n{el.hs}")

hs: 
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         -3.14159265]
 [ 0.          0.          3.14159265  0.        ]]


The property `dim` of EffectiveLindbladian is the size of square matrix `hs`.

In [79]:
print(f"dim: {el.dim}")
print(f"size of square matrix hs: {int(np.sqrt(hs.shape[0]))}")

dim: 2
size of square matrix hs: 2


### functions to check constraints
The `is_eq_constraint_satisfied()` function returns True, if and only if $e^L$ is TP(trace-preserving map), i.e. if and only if the first row of `hs` is zeros.

In [80]:
print(f"is_eq_constraint_satisfied(): {el.is_eq_constraint_satisfied()}")
print(f"is_tp(): {el.is_tp()}")
print(f"hs[0]: {el.hs[0]}")

is_eq_constraint_satisfied(): True
is_tp(): True
hs[0]: [0. 0. 0. 0.]


The `is_ineq_constraint_satisfied()` function returns True, if and only if $e^L$ is CP(Complete-Positivity-Preserving), i.e. if and only if K-part is positive semidifinite matrix.

In [81]:
import quara.utils.matrix_util as mutil

print(f"is_eq_constraint_satisfied(): {el.is_eq_constraint_satisfied()}")
print(f"is_cp(): {el.is_cp()}")
print(f"is_positive_semidefinite(): {mutil.is_positive_semidefinite(el.calc_k_mat())}")

is_eq_constraint_satisfied(): True
is_cp(): True
is_positive_semidefinite(): True


### projection functions
`calc_proj_eq_constraint()` function calculates the projection of EffectiveLindbladian on equal constraint.
This function replaces the first row of `hs` with $[0,…,0]$.

In [82]:
hs = np.array(range(16), dtype=np.float64).reshape((4, 4))
print(f"hs: \n{el.hs}")

hs: 
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         -3.14159265]
 [ 0.          0.          3.14159265  0.        ]]


In [83]:
el = EffectiveLindbladian(c_sys, hs, is_physicality_required=False)
proj_el = el.calc_proj_eq_constraint()
print(f"hs: \n{proj_el.hs}")

hs: 
[[ 0.  0.  0.  0.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]


`calc_proj_ineq_constraint()` function calculates the projection of EffectiveLindbladian with hs on inequal constraint as follows:

- Calculates H-part $H$, J-part $J$, and K-part $K$ from EffectiveLindbladian.
- Executes singular value decomposition on $K$, $K = U \Lambda U^{\dagger}$, where $\Lambda = \text{diag}[\lambda_0, \dots , \lambda_{d-1}]$, and $\lambda_{i} \in \mathbb{R}$.
- $\lambda^{\prime}_{i} := \begin{cases} \lambda_{i} & (\lambda_{i} \geq 0) \\ 0 & (\lambda_{i} < 0) \end{cases}$
- $\Lambda^{\prime} = \text{diag}[\lambda^{\prime}_0, \dots , \lambda^{\prime}_{d-1}]$
- $K^{\prime} = U \Lambda^{\prime} U^{\dagger}$
- Let $\text{HS}^{\prime}$ be a Hilbert-Schmidt matrix representation of EffectiveLindbladian from $H, J, K^{\prime}$.
- The projection of EffectiveLindbladian is EffectiveLindbladian with `hs` = $\text{HS}^{\prime}$.



In [84]:
el = EffectiveLindbladian(c_sys, hs, is_physicality_required=False)
proj_el = el.calc_proj_ineq_constraint()
print(f"hs: \n{proj_el.hs}")

16it [00:00, 296.30it/s]hs: 
[[ 8.12020183  1.71420693  5.06233752  7.16046811]
 [ 0.78579307 -5.04434853  2.49719848  1.93917625]
 [ 4.93766248  5.49719848 -2.88819756  4.92554488]
 [ 7.83953189  7.93917625  7.92554488 -0.18765574]]



### functions to transform parameters
`to_stacked_vector()` function returns a one-dimensional numpy array of all variables. This is equal to flattened `hs`.

In [85]:
print(f"to_stacked_vector(): {el.to_stacked_vector()}")
print(f"flattened hs: {el.hs.flatten()}")

to_stacked_vector(): [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
flattened hs: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]


If `on_para_eq_constraint` is True, then the first row of `hs` is equal to [0,…,0]. Thus, EffectiveLindbladian is characterized by the second and subsequent rows of `hs`.  
Therefore, `to_var()` function returns the flattened second and subsequent rows of `hs`, where `on_para_eq_constraint` is True.

In [86]:
# on_para_eq_constraint=True
el = EffectiveLindbladian(c_sys, hs, is_physicality_required=False, on_para_eq_constraint=True)
print(f"to_var(): {el.to_var()}")

to_var(): [ 4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]


In [87]:
# on_para_eq_constraint=False
el = EffectiveLindbladian(c_sys, hs, is_physicality_required=False, on_para_eq_constraint=False)
print(f"to_var(): {el.to_var()}")

to_var(): [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]


### functions to generate special objects

In [88]:
zero_el = el.generate_zero_obj()
print(f"zero: \n{zero_el.hs}")
origin_el = el.generate_origin_obj()
print(f"origin: \n{origin_el.hs}")

zero: 
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
origin: 
[[    0.     0.     0.     0.]
 [    0. -1021.     0.     0.]
 [    0.     0. -1021.     0.]
 [    0.     0.     0. -1021.]]


### supports arithmetic operations

In [89]:
hs1 = np.array(range(16), dtype=np.float64).reshape((4, 4))
el1 = EffectiveLindbladian(c_sys, hs1, is_physicality_required=False)
hs2 = np.array(range(16, 32), dtype=np.float64).reshape((4, 4))
el2 = EffectiveLindbladian(c_sys, hs2, is_physicality_required=False)

print(el1.hs)
print(el2.hs)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]
[[16. 17. 18. 19.]
 [20. 21. 22. 23.]
 [24. 25. 26. 27.]
 [28. 29. 30. 31.]]


In [90]:
print(f"sum: \n{(el1 + el2).hs}")
print(f"subtraction: \n{(el1 - el2).hs}")
print(f"right multiplication: \n{(2 * el1).hs}")
print(f"left multiplication: \n{(el1 * 2).hs}")
print(f"division: \n{(el1 / 2).hs}")

sum: 
[[16. 18. 20. 22.]
 [24. 26. 28. 30.]
 [32. 34. 36. 38.]
 [40. 42. 44. 46.]]
subtraction: 
[[-16. -16. -16. -16.]
 [-16. -16. -16. -16.]
 [-16. -16. -16. -16.]
 [-16. -16. -16. -16.]]
right multiplication: 
[[ 0.  2.  4.  6.]
 [ 8. 10. 12. 14.]
 [16. 18. 20. 22.]
 [24. 26. 28. 30.]]
left multiplication: 
[[ 0.  2.  4.  6.]
 [ 8. 10. 12. 14.]
 [16. 18. 20. 22.]
 [24. 26. 28. 30.]]
division: 
[[0.  0.5 1.  1.5]
 [2.  2.5 3.  3.5]
 [4.  4.5 5.  5.5]
 [6.  6.5 7.  7.5]]


### calc_gradient functions
Calculates gradient of EffectiveLindbladian with variable index.

In [91]:
grad_el = el.calc_gradient(0)
print(f"hs: \n{grad_el.hs}")

hs: 
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### convert_basis function
Returns `hs` converted to the specified basis.

In [92]:
from quara.objects.matrix_basis import get_comp_basis

el = generate_effective_lindbladian_from_gate_name("x", c_sys)
converted_hs = el.convert_basis(get_comp_basis())
print(f"hs: \n{converted_hs}")

hs: 
[[0.+0.j         0.+1.57079633j 0.-1.57079633j 0.+0.j        ]
 [0.+1.57079633j 0.+0.j         0.+0.j         0.-1.57079633j]
 [0.-1.57079633j 0.+0.j         0.+0.j         0.+1.57079633j]
 [0.+0.j         0.-1.57079633j 0.+1.57079633j 0.+0.j        ]]


### calc_h_mat
Calculates the matrix $H = \sum_{\alpha=1}^{d^2-1} H_\alpha B_\alpha$, with $H_\alpha = \frac{i}{2d} \text{Tr}[L^{cb}(B_\alpha \otimes I - I \otimes \overline{B_\alpha})]$.

In [93]:
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(f"calc_h_mat(): \n{el.calc_h_mat()}")

calc_h_mat(): 
[[0.        +0.j 1.57079633+0.j]
 [1.57079633+0.j 0.        +0.j]]


### calc_j_mat
Calculates the matrix $J = \sum_{\alpha=0}^{d^2-1} J_\alpha B_\alpha$, with $J_\alpha = \frac{i}{2d(1 + \delta_{0,\alpha})} \text{Tr}[L^{cb}(B_\alpha \otimes I + I \otimes \overline{B_\alpha})]$.

In [94]:
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(f"calc_j_mat(): \n{el.calc_j_mat()}")

calc_j_mat(): 
[[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


### calc_k_mat
Calculates the matrix $K$, with $(\alpha, \beta)$ entry $K_{\alpha, \beta} = \text{Tr}[L^{cb}(B_\alpha \otimes \overline{B_\beta})]$.

In [95]:
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(f"calc_k_mat(): \n{el.calc_k_mat()}")

calc_k_mat(): 
[[0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]]


### calc_h_part
Calculates H-part = $-i(H \otimes I - I \otimes \bar{H})$.

In [96]:
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(f"calc_h_part(): \n{el.calc_h_part()}")

calc_h_part(): 
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         -3.14159265]
 [ 0.          0.          3.14159265  0.        ]]


### calc_j_part
Calculates J-part = $J \otimes I + I \otimes \bar{J}$.

In [97]:
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(f"calc_j_part(): \n{el.calc_j_part()}")

calc_j_part(): 
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### calc_k_part
Calculates K-part = $\sum_{\alpha, \beta=1}^{d^2-1} K_{\alpha, \beta} B_\alpha \otimes \overline{B_\beta}$

In [98]:
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(f"calc_k_part(): \n{el.calc_k_part()}")

calc_k_part(): 
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### calc_d_part
Calculates D-part = J-part + K-part

In [99]:
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(f"calc_d_part(): \n{el.calc_d_part()}")

calc_d_part(): 
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### to_kraus_matrices
Returns Kraus matrices of EffectiveLindbladian.

In [100]:
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(f"to_kraus_matrices(): \n{el.to_kraus_matrices()}")

to_kraus_matrices(): 
[((1.7724538509055159-4.030482324789019e-33j), array([[ 5.00000000e-01+0.j , -2.01094314e-17-0.5j],
       [ 2.01094314e-17-0.5j,  5.00000000e-01+0.j ]])), ((4.0304823506953305e-33+1.7724538509055157j), array([[ 5.00000000e-01+0.00000000e+00j,  2.01094314e-17+5.00000000e-01j],
       [-2.01094314e-17+5.00000000e-01j,  5.00000000e-01+1.54074396e-33j]]))]


### to_gate
Generates Gate from EffectiveLindbladian.

In [101]:
el = generate_effective_lindbladian_from_gate_name("x", c_sys)
print(el.to_gate())

Type:
Gate

Dim:
2

HS:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-0.00000000e+00 -0.00000000e+00 -1.00000000e+00 -2.35127499e-16]
 [ 0.00000000e+00  0.00000000e+00  2.35127499e-16 -1.00000000e+00]]


### some utility functions

In [102]:
print(f"is_tp(): {el.is_tp()}")
print(f"is_cp(): {el.is_cp()}")

is_tp(): True
is_cp(): True


## effective_lindbladian_typical
`generate_effective_lindbladian_object_from_gate_name_object_name()` function in `effective_lindbladian_typical` module can easily generate objects related to State.  
The `generate_effective_lindbladian_object_from_gate_name_object_name()` function has the following arguments:

- The string that can be specified for `gate_name` can be checked by executing the `get_gate_names()`  and `get_gate_names_2qubit_asymmetric()` function. The tensor product of state_name "a", "b" is written "a_b".
- `object_name` can be the following string:
  - "hamiltonian_vec" - the vector representation of the Hamiltonian of a gate.
  - "hamiltonian_mat" - the Hamiltonian matrix of a gate.
  - "effective_lindbladian_mat" - the Hilbert-Schmidt representation matrix of an effective lindbladian.
  - "effective_lindbladian" - EffectiveLindbladian object.
- `c_sys` - CompositeSystem of objects related to EffectiveLindbladian. Specify when `object_name` is "effective_lindbladian".
- `is_physicality_required` - Whether the generated object is physicality required, by default True.

In [103]:
from quara.objects.gate_typical import (
    get_gate_names,
    get_gate_names_2qubit_asymmetric,
)
from quara.objects.effective_lindbladian_typical import generate_effective_lindbladian_object_from_gate_name_object_name

print(f"get_gate_names(): \n{get_gate_names()}")
print(f"get_gate_names_2qubit_asymmetric(): \n{get_gate_names_2qubit_asymmetric()}")

get_gate_names(): 
['identity', 'x90', 'x180', 'x', 'y90', 'y180', 'y', 'z90', 'z180', 'z', 'phase', 'phase_daggered', 'piover8', 'piover8_daggered', 'hadamard', 'cx', 'cz', 'swap', 'zx90', 'zz90', 'toffoli', 'fredkin', '01x90', '01y90', '01z90', '12x90', '12y90', '12z90', '02x90', '02y90', '02z90', '01x180', '01y180', '01z180', '12x180', '12y180', '12z180', '02x180', '02y180', '02z180', 'i01x90', 'i01x180', 'i01y90', 'i01y180', 'i01z90', 'i01z180', 'i12x90', 'i12x180', 'i12y90', 'i12y180', 'i12z90', 'i12z180', 'i02x90', 'i02x180', 'i02y90', 'i02y180', 'i02z90', 'i02z180', '01xi90', '01xi180', '01x01x90', '01x01x180', '01x01y90', '01x01y180', '01x01z90', '01x01z180', '01x12x90', '01x12x180', '01x12y90', '01x12y180', '01x12z90', '01x12z180', '01x02x90', '01x02x180', '01x02y90', '01x02y180', '01x02z90', '01x02z180', '01yi90', '01yi180', '01y01x90', '01y01x180', '01y01y90', '01y01y180', '01y01z90', '01y01z180', '01y12x90', '01y12x180', '01y12y90', '01y12y180', '01y12z90', '01y12z180', '01

### object_name = "hamiltonian_vec"

In [104]:
vec = generate_effective_lindbladian_object_from_gate_name_object_name("x", "hamiltonian_vec")
print(vec)

[-2.22144147  2.22144147  0.          0.        ]


### object_name = “hamiltonian_mat”

In [105]:
mat = generate_effective_lindbladian_object_from_gate_name_object_name("x", "hamiltonian_mat")
print(mat)

[[-1.57079633+0.j  1.57079633+0.j]
 [ 1.57079633+0.j -1.57079633+0.j]]


### object_name = “effective_lindbladian_mat”

In [106]:
mat = generate_effective_lindbladian_object_from_gate_name_object_name("x", "effective_lindbladian_mat")
print(mat)

[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         -3.14159265]
 [ 0.          0.          3.14159265  0.        ]]


### object_name = “effective_lindbladian”

In [107]:
c_sys = generate_composite_system("qubit", 1)
el = generate_effective_lindbladian_object_from_gate_name_object_name("x", "effective_lindbladian", c_sys=c_sys)
print(el)

Type:
EffectiveLindbladian

Dim:
2

HS:
[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         -3.14159265]
 [ 0.          0.          3.14159265  0.        ]]


## MProcess (Measurement Process)
MProcess $M = \{ M_x \}_{x=0,\dots,m-1}$ maps State $\rho \mapsto \{ \rho_x \}_{x=0,\dots,m-1}$, where $\rho_x = \frac{M_x(\rho)}{\text{Tr}[M_x(\rho)]}$, with probability $p(x) = \text{Tr}[M_x(\rho)] = \text{Tr}[\Pi_x\rho]$.  
Each $M_x$ is a linear trace-preserving and completely positive (L-TPCP) map on the space of quantum states.  
The Hilbert-Schmidt matrix representations of these L-TPCP maps is denote `hss` in Quara. `hss` is a list of 2-dimensional numpy array.  

The property `mode_sampling` of MProcess is whether to sample to determine one Hilbert-Schumidt matrix with `compose_qoperations()` function.  
The property `random_seed_or_generator` of MProcess is the random seed or numpy.random.Generator to sample.

Generate from `mprocess_typical` module by specifying CompositeSystem and state mprocess (ex. “z-type1”).

In [108]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.mprocess_typical import generate_mprocess_from_name

c_sys = generate_composite_system("qubit", 1)
mprocess = generate_mprocess_from_name(c_sys, "z-type1")
print(mprocess)

16it [00:00, 666.62it/s]Type:
MProcess

Dim:
2

HSs:
[array([[0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5]]), array([[ 0.5,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [-0.5,  0. ,  0. ,  0.5]])]

ModeSampling:
False



Generate MProcess object directly using CompositeSystem and a numpy array.

In [109]:
import numpy as np

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.mprocess import MProcess

basis = get_normalized_pauli_basis(1)
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])
hss = [
    np.array([[0.5, 0, 0, 0.5], [0, 0, 0, 0], [0, 0, 0, 0], [0.5, 0, 0, 0.5]]),
    np.array([[0.5, 0, 0, -0.5], [0, 0, 0, 0], [0, 0, 0, 0], [-0.5, 0, 0, 0.5]]),
]
mprocess = MProcess(c_sys, hss)
print(mprocess)

16it [00:00, 516.14it/s]Type:
MProcess

Dim:
2

HSs:
[array([[0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5]]), array([[ 0.5,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [-0.5,  0. ,  0. ,  0.5]])]

ModeSampling:
False



### specific properties
The property `hss` of MProcess is a numpy array specified by the constructor argument `hss`.

In [110]:
mprocess = MProcess(c_sys, hss)
print(f"hss: \n{mprocess.hss}")

hss: 
[array([[0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5]]), array([[ 0.5,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [-0.5,  0. ,  0. ,  0.5]])]


The `hs()` function returns a numpy array specified by the constructor argument `hss`.

In [111]:
mprocess = MProcess(c_sys, hss)
print(f"hs(0): \n{mprocess.hs(0)}")
print(f"hs(1): \n{mprocess.hs(1)}")

hs(0): 
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]
hs(1): 
[[ 0.5  0.   0.  -0.5]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [-0.5  0.   0.   0.5]]


The property `dim` of MProcess is the size of square matrices `hss`.

In [112]:
print(f"dim: {mprocess.dim}")
print(f"size of square matrices hss: {int(np.sqrt(hss[0].shape[0]))}")

dim: 2
size of square matrices hss: 2


The property `num_outcomes` of MProcess is the number of `hss`.

In [113]:
print(f"num_outcomes: {mprocess.num_outcomes}")
print(f"number of hss: {len(mprocess.hss)}")

num_outcomes: 2
number of hss: 2


The property `mode_sampling` of MProcess is the mode of sampling.

In [114]:
print(mprocess.mode_sampling)

False


### functions to check constraints
The `is_eq_constraint_satisfied()` function returns True, if and only if the sum of `hss` is TP(trace-preserving map).

In [115]:
print(f"is_eq_constraint_satisfied(): {mprocess.is_eq_constraint_satisfied()}")
print(f"is_sum_tp(): {mprocess.is_sum_tp()}")

is_eq_constraint_satisfied(): True
is_sum_tp(): True


The `is_ineq_constraint_satisfied()` function returns True, if and only if all matrices of `hss` are CP(Complete-Positivity-Preserving), i.e. if and only if all Choi matrices of `hss` are positive semidifinite matrices.

In [116]:
print(f"is_ineq_constraint_satisfied(): {mprocess.is_ineq_constraint_satisfied()}")
print(f"is_cp(): {mprocess.is_cp()}")

is_ineq_constraint_satisfied(): True
is_cp(): True


### projection functions
`calc_proj_eq_constraint()` function calculates the projection of MProcess on equal constraint.
Let `hss` of MProcess be $\{ M_0, \dots, M_{m-1}\}$, where $m$ is `num_outcomes` of MProcess.
When MProcess object satifies on equal constraint, the first row of $\sum_{x=0}^{m-1} M_x$ is equal to $[1, 0, \dots, 0]$.  
Therefore, `calc_proj_eq_constraint()` function calculates the projection of MProcess as follows:

- $\text{vec} :=$ the first row of $\sum_{x=0}^{m-1} M_x$
- for each $x$, calculates $M^{\prime}_x = M_x - \frac{1}{m}\begin{bmatrix} \text{vec} \\ 0  \\ \vdots \\ 0 \end{bmatrix} + \frac{1}{m}\begin{bmatrix} 1 & 0 & \dots & 0 \\ 0 \\ \vdots & & \huge{0} \\ 0 \end{bmatrix}$
- The projection of MProcess is $\{ M^{\prime}_x \}_{x=0}^{m-1}$.

In [117]:
hss = [
    np.array(range(16), dtype=np.float64).reshape((4, 4)),
    np.array(range(16, 32), dtype=np.float64).reshape((4, 4)),
]

In [118]:
mprocess = MProcess(c_sys, hss, is_physicality_required=False)
proj_mprocess = mprocess.calc_proj_eq_constraint()
print(f"hss: \n{proj_mprocess.hss}")

hss: 
[array([[-7.5, -8. , -8. , -8. ],
       [ 4. ,  5. ,  6. ,  7. ],
       [ 8. ,  9. , 10. , 11. ],
       [12. , 13. , 14. , 15. ]]), array([[ 8.5,  8. ,  8. ,  8. ],
       [20. , 21. , 22. , 23. ],
       [24. , 25. , 26. , 27. ],
       [28. , 29. , 30. , 31. ]])]


`calc_proj_ineq_constraint()` function calculates the projection of MProcess with `hss` $\{ M_x \}_{x=0}^{m-1}$ on inequal constraint as follows:

- For each $x$, calculate the following:
  - Let $\text{Choi}_x$ be Choi matrix of $M_x$
  - Executes singular value decomposition on $\text{Choi}_x$, $\text{Choi}_x = U \Lambda U^{\dagger}$, where $\Lambda = \text{diag}[\lambda_0, \dots , \lambda_{d-1}]$, and $\lambda_{i} \in \mathbb{R}$.
  - $\lambda^{\prime}_{i} := \begin{cases} \lambda_{i} & (\lambda_{i} \geq 0) \\ 0 & (\lambda_{i} < 0) \end{cases}$
  - $\Lambda^{\prime} = \text{diag}[\lambda^{\prime}_0, \dots , \lambda^{\prime}_{d-1}]$
  - $\text{Choi}^{\prime}_x = U \Lambda^{\prime} U^{\dagger}$
  - Let $M^{\prime}_x$ be Hilbert-Schmidt matrix representation of $\text{Choi}^{\prime}_x$
- The projection of MProcess is $\{ M^{\prime}_x \}_{x=0}^{m-1}$.

In [119]:
mprocess = MProcess(c_sys, hss, is_physicality_required=False)
proj_mprocess = mprocess.calc_proj_ineq_constraint()
print(f"hss: \n{proj_mprocess.hss}")

hss: 
[array([[15.84558996,  4.43570942,  5.29265833,  6.14960724],
       [ 2.63097854,  2.34702553,  3.08437192,  3.44443746],
       [ 4.98440409,  4.50900796,  4.73510284,  5.71575945],
       [ 7.33782964,  6.29370952,  7.14039548,  7.60980059]]), array([[47.32687829, 20.14755601, 21.1037168 , 22.0598776 ],
       [17.5667235 , 12.54854821, 13.24883203, 13.79656536],
       [20.93808076, 15.04184745, 15.53818711, 16.33962776],
       [24.30943803, 17.3825962 , 18.13264319, 18.73013967]])]


### functions to transform parameters
`to_stacked_vector()` function returns a one-dimensional numpy array of all variables. This is equal to a concatenated vector of flattened matrices of `hss`.

In [120]:
print(f"to_stacked_vector(): \n{mprocess.to_stacked_vector()}")
print(f"hss: \n{mprocess.hss}")

to_stacked_vector(): 
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]
hss: 
[array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.],
       [12., 13., 14., 15.]]), array([[16., 17., 18., 19.],
       [20., 21., 22., 23.],
       [24., 25., 26., 27.],
       [28., 29., 30., 31.]])]


Let `hss` of MProcess be $\{ M_0, \dots, M_{m-1}\}$, where $m$ is `num_outcomes` of MProcess. Let $\text{vec}(M_x) := |M_x\rangle\rangle$. Let $\text{vec}(\tilde{M}_x) $ be a flattened vector of the second and subsequent rows of matrix $M_x$.  
If `on_para_eq_constraint` is True, then the first row of $\sum_{x=0}^{m-1} M_x$ is equal to $[1, 0, \dots, 0]$.  
Therefore, `to_var()` function returns a concatenated vector of $\text{vec}(M_0), \dots, \text{vec}(M_{m-2}), \text{vec}(\tilde{M}_{m-1})$, where `on_para_eq_constraint` is True.

In [121]:
# on_para_eq_constraint=True
mprocess = MProcess(c_sys, hss, is_physicality_required=False, on_para_eq_constraint=True)
print(f"to_var(): \n{mprocess.to_var()}")

to_var(): 
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]


In [122]:
# on_para_eq_constraint=False
mprocess = MProcess(c_sys, hss, is_physicality_required=False, on_para_eq_constraint=False)
print(f"to_var(): \n{mprocess.to_var()}")

to_var(): 
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]


### functions to generate special objects

In [123]:
zero_mprocess = mprocess.generate_zero_obj()
print(f"zero: \n{zero_mprocess.hss}")
origin_mprocess = mprocess.generate_origin_obj()
print(f"origin: \n{origin_mprocess.hss}")

zero: 
[array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])]
origin: 
[array([[0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]]), array([[0.5, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])]


### supports arithmetic operations

In [124]:
hs11 = np.array(range(16), dtype=np.float64).reshape((4, 4))
hs12 = np.array(range(16, 32), dtype=np.float64).reshape((4, 4))
mprocess1 = MProcess(c_sys, [hs11, hs12], is_physicality_required=False)
hs21 = np.array(range(32, 48), dtype=np.float64).reshape((4, 4))
hs22 = np.array(range(48, 64), dtype=np.float64).reshape((4, 4))
mprocess2 = MProcess(c_sys, [hs21, hs22], is_physicality_required=False)

print(mprocess1.hss)
print(mprocess2.hss)

[array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.],
       [12., 13., 14., 15.]]), array([[16., 17., 18., 19.],
       [20., 21., 22., 23.],
       [24., 25., 26., 27.],
       [28., 29., 30., 31.]])]
[array([[32., 33., 34., 35.],
       [36., 37., 38., 39.],
       [40., 41., 42., 43.],
       [44., 45., 46., 47.]]), array([[48., 49., 50., 51.],
       [52., 53., 54., 55.],
       [56., 57., 58., 59.],
       [60., 61., 62., 63.]])]


In [125]:
print(f"sum: \n{(mprocess1 + mprocess2).hss}")
print(f"subtraction: \n{(mprocess1 - mprocess2).hss}")
print(f"right multiplication: \n{(2 * mprocess1).hss}")
print(f"left multiplication: \n{(mprocess1 * 2).hss}")
print(f"division: \n{(mprocess1 / 2).hss}")

sum: 
[array([[32., 34., 36., 38.],
       [40., 42., 44., 46.],
       [48., 50., 52., 54.],
       [56., 58., 60., 62.]]), array([[64., 66., 68., 70.],
       [72., 74., 76., 78.],
       [80., 82., 84., 86.],
       [88., 90., 92., 94.]])]
subtraction: 
[array([[-32., -32., -32., -32.],
       [-32., -32., -32., -32.],
       [-32., -32., -32., -32.],
       [-32., -32., -32., -32.]]), array([[-32., -32., -32., -32.],
       [-32., -32., -32., -32.],
       [-32., -32., -32., -32.],
       [-32., -32., -32., -32.]])]
right multiplication: 
[array([[ 0.,  2.,  4.,  6.],
       [ 8., 10., 12., 14.],
       [16., 18., 20., 22.],
       [24., 26., 28., 30.]]), array([[32., 34., 36., 38.],
       [40., 42., 44., 46.],
       [48., 50., 52., 54.],
       [56., 58., 60., 62.]])]
left multiplication: 
[array([[ 0.,  2.,  4.,  6.],
       [ 8., 10., 12., 14.],
       [16., 18., 20., 22.],
       [24., 26., 28., 30.]]), array([[32., 34., 36., 38.],
       [40., 42., 44., 46.],
       [48., 50

### calc_gradient functions
Calculates gradient of MProcess with variable index.

In [126]:
grad_mprocess = mprocess.calc_gradient(0)
print(f"hss: \n{grad_mprocess.hss}")

hss: 
[array([[1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])]


### convert_basis function
Returns `hss` converted to the specified basis.

In [127]:
from quara.objects.matrix_basis import get_comp_basis

mprocess = generate_mprocess_from_name(c_sys, "z-type1")
converted_hss = mprocess.convert_basis(get_comp_basis())
print(f"hss: \n{converted_hss}")

hss: 
[array([[1.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, 6.22328532e-19+0.j],
       [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
       [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
       [3.25176795e-17+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, 5.97792087e-34+0.j]]), array([[5.97792087e-34+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, 3.25176795e-17+0.j],
       [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
       [0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j],
       [6.22328532e-19+0.j, 0.00000000e+00+0.j, 0.00000000e+00+0.j, 1.00000000e+00+0.j]])]


### to_choi_matrix
Returns Choi matrix of the specified index of `hss`.

In [128]:
mprocess = generate_mprocess_from_name(c_sys, "z-type1")
print(f"to_choi_matrix(0): \n{mprocess.to_choi_matrix(0)}")
print(f"to_choi_matrix_with_dict(0): \n{mprocess.to_choi_matrix_with_dict(0)}")
print(f"to_choi_matrix(0): \n{mprocess.to_choi_matrix(0)}")

to_choi_matrix(0): 
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
to_choi_matrix_with_dict(0): 
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
to_choi_matrix(0): 
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


### to_kraus_matrices
Returns Kraus matrices of the specified index of `hss`.

In [129]:
mprocess = generate_mprocess_from_name(c_sys, "z-type1")
print(f"to_kraus_matrices(0): \n{mprocess.to_kraus_matrices(0)}")

to_kraus_matrices(0): 
[array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])]


### to_process_matrix
Returns process matrix of the specified index of `hss`.

In [130]:
mprocess = generate_mprocess_from_name(c_sys, "z-type1")
print(f"to_process_matrix(0): \n{mprocess.to_process_matrix(0)}")

to_process_matrix(0): 
[[1.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 6.22328532e-19+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 3.25176795e-17+0.j 0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j 5.97792087e-34+0.j]]


### to_povm
Generates Povm from MProcess.

In [131]:
mprocess = generate_mprocess_from_name(c_sys, "z-type1")
print(mprocess.to_povm())

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]


### some utility functions

In [132]:
print(f"is_sum_tp(): {mprocess.is_sum_tp()}")
print(f"is_cp(): {mprocess.is_cp()}")

is_sum_tp(): True
is_cp(): True


## mprocess_typical
`generate_mprocess_object_from_mprocess_name_object_name()` function in `mprocess_typical` module can easily generate objects related to MProcess.  
The `generate_mprocess_object_from_mprocess_name_object_name()` function has the following arguments:

- The string that can be specified for `mprocess_name` can be checked by executing the `get_mprocess_names_type1()` and `get_mprocess_names_type2()` functions. The tensor product of state_name "a", "b" is written "a_b".
- `object_name` can be the following string:
  - "set_pure_state_vectors" - The set of pure state vectors of MProcess.
  - "set_kraus_matrices" - The set of Kraus matrices of MProcess.
  - "hss" - The list of Hilbert-Schmidt matrix representations of MProcess.
  - "mprocess" - MProcess object.
- `c_sys` - CompositeSystem of objects related to MProcess. Specify when `object_name` is "hss" and "mprocess".
- `is_physicality_required` - Whether the generated object is physicality required, by default True.

In [133]:
from quara.objects.mprocess_typical import (
    get_mprocess_names_type1,
    get_mprocess_names_type2,
    generate_mprocess_object_from_mprocess_name_object_name,
)

print(f"get_mprocess_names_type1(): \n{get_mprocess_names_type1()}")
print(f"get_mprocess_names_type2(): \n{get_mprocess_names_type2()}")

get_mprocess_names_type1(): 
['x-type1', 'y-type1', 'z-type1', 'bell-type1', 'z3-type1', 'z2-type1', 'xxparity-type1', 'zzparity-type1']
get_mprocess_names_type2(): 
['x-type2', 'y-type2', 'z-type2', 'z3-type2', 'z2-type2']


### object_name = "set_pure_state_vectors"

In [134]:
vecs = generate_mprocess_object_from_mprocess_name_object_name("z-type1", "set_pure_state_vectors")
print(vecs)

[[array([1.+0.j, 0.+0.j])], [array([0.+0.j, 1.+0.j])]]


### object_name = "set_kraus_matrices"

In [135]:
matrices = generate_mprocess_object_from_mprocess_name_object_name("z-type1", "set_kraus_matrices")
print(matrices)

[[array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])], [array([[0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])]]


### object_name = "hss"

In [136]:
c_sys = generate_composite_system("qubit", 1)
hss = generate_mprocess_object_from_mprocess_name_object_name("z-type1", "hss", c_sys=c_sys)
print(hss)

[array([[0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5]]), array([[ 0.5,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [-0.5,  0. ,  0. ,  0.5]])]


### object_name = "mprocess"

In [137]:
c_sys = generate_composite_system("qubit", 1)
mprocess = generate_mprocess_object_from_mprocess_name_object_name("z-type1", "mprocess", c_sys=c_sys)
print(mprocess)

16it [00:00, 533.31it/s]Type:
MProcess

Dim:
2

HSs:
[array([[0.5, 0. , 0. , 0.5],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0.5]]), array([[ 0.5,  0. ,  0. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [-0.5,  0. ,  0. ,  0.5]])]

ModeSampling:
False



## MultinomialDistribution
MultinomialDistribution is a joint probability distribution of a discrete random variables $(x_0, \dots, x_{n-1})$, with events of $x_i$ are $A_i$ and $\sum_{a_0 \in A_0} \cdots \sum_{a_{n-1} \in A_{n-1}} p(x_0=a_0, \dots, x_{n-1}=a_{n-1}) = 1$.  
The property `ps` of MultinomialDistribution is a flattened numpy array.  
If $s_i$ be sizes of $A_i$, then the property `shape` of MultinomialDistribution is $(s_0, \dots, s_{n-1})$.

Exmaple.  
Consider the following joint probability distribution:

| x_0 \\ x_1 | 0 | 1 | 2 |
| --- | --- | --- | --- |
| 0 | 0.0 | 0.10 | 0.15 |
| 1 | 0.20 | 0.25 | 0.30 |

Then `ps` = $[0.0, 0.10, 0.15, 0.20, 0.25, 0.30]$, and `shape` = $[2, 3]$.

Generate MultinomialDistribution object directly.

In [138]:
from quara.objects.multinomial_distribution import MultinomialDistribution

ps = np.array([0.0, 0.10, 0.15, 0.20, 0.25, 0.30], dtype=np.float64)
shape = (2, 3)
dist = MultinomialDistribution(ps, shape=shape)
print(dist)

shape = (2, 3)
ps = [0.   0.1  0.15 0.2  0.25 0.3 ]


If `shape` is omitted, the number of random variables are assumed to be one.

In [139]:
dist = MultinomialDistribution(ps)
print(dist)

shape = (6,)
ps = [0.   0.1  0.15 0.2  0.25 0.3 ]


### specific properties
The property `ps` of MultinomialDistribution is a one-dimensional numpy array specified by the constructor argument `ps`.

In [140]:
dist = MultinomialDistribution(ps, shape=shape)
print(f"ps: {dist.ps}")

ps: [0.   0.1  0.15 0.2  0.25 0.3 ]


The property `shape` of MultinomialDistribution is a tuple of int by the constructor argument `shape`.

In [141]:
dist = MultinomialDistribution(ps, shape=shape)
print(f"shape: {dist.shape}")

shape: (2, 3)


### marginalize
Returns MultinomialDistribution corresponding to marginal probability.  
The marginal probability of variable ``outcome_indices_remain``.

Exmaple.  
Consider the following joint probability distribution:

| x_0 \\ x_1 | 0 | 1 | 2 |
| --- | --- | --- | --- |
| 0 | 0.0 | 0.10 | 0.15 |
| 1 | 0.20 | 0.25 | 0.30 |

Let ``outcome_indices_remain`` = $[0] (= [x_0])$.  
Then marginal probability is $p(x_0 = 0) = 0.25, p(x_0 = 1) = 0.75$.

In [142]:
dist = MultinomialDistribution(ps, shape=shape)
marginalized_dist = dist.marginalize([0])
print(f"marginalize([0]): \n{marginalized_dist}")

marginalize([0]): 
shape = (2,)
ps = [0.25 0.75]


### conditionalize
Returns MultinomialDistribution corresponding to marginal probability.  
The marginal probability of variable ``outcome_indices_remain``.

Exmaple.  
Consider the following joint probability distribution:

| x_0 \\ x_1 | 0 | 1 | 2 | sum |
| --- | --- | --- | --- | --- |
| 0 | 0.0 | 0.10 | 0.15 | 0.25 |
| 1 | 0.20 | 0.25 | 0.30 | 0.75 |

Let `conditional_variable_indices` = $[0] (= [x_0])$, and `conditional_variable_values` = $[1]$ (i.e. $x_0 = 1$).  
Then marginal probabilities are

- $p(x_1 = 0|x_0 = 1) = p(x_0 = 1, x_1 = 0) / p(x_0 = 1) = 0.20 / 0.75 \fallingdotseq 0.27$,
- $p(x_1 = 1|x_0 = 1) = p(x_0 = 1, x_1 = 1) / p(x_0 = 1) = 0.25 / 0.75 \fallingdotseq 0.33$,
- $p(x_1 = 2|x_0 = 1) = p(x_0 = 1, x_1 = 2) / p(x_0 = 1) = 0.30 / 0.75 \fallingdotseq 0.40$.

In [143]:
dist = MultinomialDistribution(ps, shape=shape)
conditionalized_dist = dist.conditionalize([0], [1])
print(f"conditionalize([0], [1]): \n{conditionalized_dist}")

conditionalize([0], [1]): 
shape = (3,)
ps = [0.26666667 0.33333333 0.4       ]


### execute_random_sampling
Returns results of random sampling.  
The first argument `num` is the number of trials per execution unit. The second argument `size` is the number of execution.  
Optional argument `random_generator` is seed(int) or np.random.Generator used for sampling.  

Examle.  
`num` = 100, `size` = 3

In [144]:
dist = MultinomialDistribution(ps, shape=shape)
samples = dist.execute_random_sampling(100, 3)
print(f"execute_random_sampling(): \n{samples}")

execute_random_sampling(): 
[array([ 0, 13, 14, 21, 22, 30]), array([ 0, 15, 15, 12, 23, 35]), array([ 0,  8, 21, 17, 24, 30])]


## StateEnsemble
StateEnsemble is a list of State with corresponding probabilities(MultinomialDistribution).  
The property `states` is a list of State.
The property `ps` is probabilities.


Example.  
If StateEnsemble is a ensemble of $(1/2, z_0)$ and $(1/2, z_1)$, then `states` = $[z_0, z_1]$ and `ps` = MultinomialDistribution of $[1/2, 1/2]$.

The methods for generating a StateEnsemble includes the following:

- Generate from `state_ensemble_typical` module
- Generate StateEnsemble object directly

Generate from `state_ensemble_typical` module by specifying CompositeSystem and state name (ex. “z0”).

In [145]:
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.state_ensemble_typical import generate_state_ensemble_from_name

c_sys = generate_composite_system("qubit", 1)
ensemble = generate_state_ensemble_from_name(c_sys, "z0")
print(ensemble)

Type:
StateEnsemble

States:
states[0]: [0.70710678 0.         0.         0.70710678]
states[1]: [ 0.70710678  0.          0.         -0.70710678]

MultinomialDistribution:
shape = (2,)
ps = [1. 0.]


Generate StateEnsemble object directly using a list of State and MultinomialDistribution.

In [146]:
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import State
from quara.objects.multinomial_distribution import MultinomialDistribution
from quara.objects.state_ensemble import StateEnsemble

basis = get_normalized_pauli_basis(1)
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])
vec1 = np.array([1, 0, 0, 1]) / np.sqrt(2)
state1 = State(c_sys, vec1)
vec2 = np.array([1, 0, 0, -1]) / np.sqrt(2)
state2 = State(c_sys, vec2)

ps = [1.0, 0.0]
dist = MultinomialDistribution(ps)
ensemble = StateEnsemble([state1, state2], dist)
print(ensemble)

Type:
StateEnsemble

States:
states[0]: [0.70710678 0.         0.         0.70710678]
states[1]: [ 0.70710678  0.          0.         -0.70710678]

MultinomialDistribution:
shape = (2,)
ps = [1. 0.]


### specific properties
The property vec of `states` is a list of State specified by the constructor argument `states`.

In [147]:
ensemble = StateEnsemble([state1, state2], dist)
print(f"states: \n{ensemble.states}")

states: 
[<quara.objects.state.State object at 0x0000020373D8DD48>, <quara.objects.state.State object at 0x000002036E091088>]


The `state()` function returns a State specified by the constructor argument `state`.

In [148]:
ensemble = StateEnsemble([state1, state2], dist)
print(f"state(0): \n{ensemble.state(0)}")
print(f"state(1): \n{ensemble.state(1)}")

state(0): 
Type:
State

Dim:
2

Vec:
[0.70710678 0.         0.         0.70710678]
state(1): 
Type:
State

Dim:
2

Vec:
[ 0.70710678  0.          0.         -0.70710678]


The property vec of `prob_dist` is a MultinomialDistribution specified by the constructor argument `prob_dist`.

In [149]:
ensemble = StateEnsemble([state1, state2], dist)
print(f"prob_dist: {ensemble.prob_dist}")

prob_dist: shape = (2,)
ps = [1. 0.]


### other functions
Many functions that other subclasses of QOperation have are not supported in StateEnsemble.
If an unsupported function is executed, a NotImplementedError will be raised.

## state_ensemble_typical
`generate_state_ensemble_object_from_state_ensemble_name_object_name()` function in `state_ensemble_typical` module can easily generate objects related to MProcess.  
The `generate_state_ensemble_object_from_state_ensemble_name_object_name()` function has the following arguments:

- The string that can be specified for `mprocess_name` can be checked by executing the `get_state_ensemble_names()` function. The tensor product of state_ensemble_names "a", "b" is written "a_b".
- `object_name` can be the following string:
  - "state_ensemble" - StateEnsemble object.
- `c_sys` - CompositeSystem of objects related to MProcess. Specify when `object_name` is "state_ensemble".
- `is_physicality_required` - Whether the generated object is physicality required, by default True.

In [150]:
from quara.objects.state_ensemble_typical import (
    get_state_ensemble_names,
    generate_state_ensemble_object_from_state_ensemble_name_object_name,
)

print(f"get_state_ensemble_names(): \n{get_state_ensemble_names()}")

get_state_ensemble_names(): 
['x0', 'x1', 'y0', 'y1', 'z0', 'z1', 'a']


### object_name = “state_ensemble”

In [151]:
c_sys = generate_composite_system("qubit", 1)
ensemble = generate_state_ensemble_object_from_state_ensemble_name_object_name("z0", "state_ensemble", c_sys=c_sys)
print(ensemble)

Type:
StateEnsemble

States:
states[0]: [0.70710678 0.         0.         0.70710678]
states[1]: [ 0.70710678  0.          0.         -0.70710678]

MultinomialDistribution:
shape = (2,)
ps = [1. 0.]
